In [9]:
# ! pip install pandas numpy tensorflow-gpu scikit-learn plotly matplotlib
# ! pip install nbformat
# ! pip install --upgrade nbformat
# ! pip install tensorflow[and-cuda]

In [5]:
import pandas as pd
import numpy as np
import tensorflow as tf
from datetime import datetime as dt

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix

from tensorflow.keras.layers import Dense, Input, Dropout, Rescaling, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import Precision, Recall

from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px

import warnings 
warnings.filterwarnings('ignore')

In [6]:
# Check if GPUs are available for training 
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [25]:
BATCH_SIZE=32

#### Train/test sets

In [26]:
# Train set : CIFAKE/train
# Class FAKE : CIFAKE/train/FAKE
# Class REAL : CIFAKE/train/REAL
train_set = tf.keras.utils.image_dataset_from_directory(
    'CIFAKE/train', 
    seed=42,
    image_size=(32,32),
    batch_size=BATCH_SIZE
)

# Test set : CIFAKE > test
# Class FAKE : CIFAKE/test/FAKE
# Class REAL : CIFAKE/test/REAL
validation_set = tf.keras.utils.image_dataset_from_directory(
    'CIFAKE/test', 
    seed=42,
    image_size=(32,32),
    batch_size=BATCH_SIZE
)

Found 100000 files belonging to 2 classes.
Found 20000 files belonging to 2 classes.


In [27]:
class_names = train_set.class_names
print(f'Training classes: {class_names}')

class_names = validation_set.class_names
print(f'Validation classes: {class_names}')

Training classes: ['FAKE', 'REAL']
Validation classes: ['FAKE', 'REAL']


#### Image normalisation

In [28]:
# Image normalisation

#### Build model

In [29]:
# Build model
def CNN_model01():
    model = Sequential([
        Rescaling(1./255),
        Conv2D(32, 3, activation='relu'),
        MaxPooling2D(),
        Flatten(),

        Dense(64, activation='relu'),
        Dense(1, activation='sigmoid')
    ])

    model.compile( optimizer='adam', loss=BinaryCrossentropy(),
                  metrics = ['accuracy', Precision(), Recall()]                  
                  )

    model.build(input_shape=(None, 32, 32, 3))
    return model

In [30]:
CNN_model01().summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ rescaling_2 (Rescaling)         │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 30, 30, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 15, 15, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 7200)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 64)             │       460,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 461,825 (1.76 MB)

 Trainable params: 461,825 (1.76 MB)

 Non-trainable params: 0 (0.00 B)

#### Train model

In [31]:
time_start = dt.now()
print(f'Start training, time: {time_start.time()}')

model = CNN_model01()

history = model.fit(
  train_set,
  validation_data=validation_set,
  epochs=5,
  verbose=1
)

print(f'Time elapsed: {dt.now() - time_start}')

Start training, time: 13:48:23.595738
Epoch 1/5
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 13s 4ms/step - accuracy: 0.7646 - loss: 0.4903 - precision_3: 0.8137 - recall_3: 0.6760 - val_accuracy: 0.8777 - val_loss: 0.2929 - val_precision_3: 0.8523 - val_recall_3: 0.9138
Epoch 2/5
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 12s 4ms/step - accuracy: 0.8776 - loss: 0.2907 - precision_3: 0.8660 - recall_3: 0.8928 - val_accuracy: 0.8719 - val_loss: 0.3046 - val_precision_3: 0.8167 - val_recall_3: 0.9590
Epoch 3/5
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 12s 4ms/step - accuracy: 0.8938 - loss: 0.2576 - precision_3: 0.8840 - recall_3: 0.9061 - val_accuracy: 0.8826 - val_loss: 0.2794 - val_precision_3: 0.8324 - val_recall_3: 0.9580
Epoch 4/5
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 12s 4ms/step - accuracy: 0.9025 - loss: 0.2392 - precision_3: 0.8947 - recall_3: 0.9118 - val_accuracy: 0.9043 - val_loss: 0.2408 - val_precision_3: 0.9012 - val_recall_3: 0.9082
Epoch 5/5
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 13s 4ms/step - accuracy: 0.9090 - loss: 0

#### Visualisation

In [32]:
pd.DataFrame(history.history)

,accuracy,loss,precision_3,recall_3,val_accuracy,val_loss,val_precision_3,val_recall_3
0,0.82645,0.391705,0.835640,0.81276,0.87770,0.292900,0.852266,0.9138
1,0.88416,0.281312,0.873203,0.89884,0.87190,0.304577,0.816726,0.9590
2,0.89675,0.254353,0.887656,0.90848,0.88255,0.279404,0.832392,0.9580
3,0.90453,0.237105,0.896824,0.91424,0.90430,0.240756,0.901171,0.9082
4,0.90836,0.228962,0.901423,0.91700,0.88940,0.267301,0.844907,0.9539


In [33]:
hist = history.history

train_col = 'precision_1'
val_col = f'val_{train_col}'


fig = make_subplots(rows=1, cols=2, subplot_titles=("Loss over epochs", f"{train_col} over epochs"))

# Loss
fig.add_trace( go.Scatter(x=list(range(len(hist['loss']))), y=hist['loss'], mode='lines', name='Training Loss'), row=1, col=1 )
fig.add_trace( go.Scatter(x=list(range(len(hist['val_loss']))), y=hist['val_loss'], mode='lines', name='Validation Loss'), row=1, col=1 )

fig.add_trace(  go.Scatter(x=list(range(len(hist[train_col]))), y=hist[train_col],  mode='lines', name=f'Training {train_col}'),  row=1, col=2 )
fig.add_trace( go.Scatter(x=list(range(len(hist[val_col]))), y=hist[val_col], mode='lines', name=f'Validation {train_col}'), row=1, col=2)

fig.update_xaxes(title_text="Epochs", row=1, col=1)
fig.update_xaxes(title_text="Epochs", row=1, col=2)
fig.update_yaxes(title_text="Loss", row=1, col=1)
fig.update_yaxes(title_text=f"{train_col}", row=1, col=2)
fig.update_layout(
    # title_text="Training and Validation Metrics over Epochs",
    showlegend=True,
    margin=dict(l=10, r=10, b=10, t=30),
    width=1000, height=400
)

KeyError: 'precision_1'

In [ ]:
# Confusion matrix
# conf_matrix = confusion_matrix(y_test, y_pred_classes)

#### Class activation map